In [40]:
#import need
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
import math
import elm
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
plt.rcParams['figure.figsize'] = (12,5)


In [42]:
data_climate_test_train = pd.read_csv("fortaleza_junho_julho.csv")
data_climate_test_train.head()

,codigo_estacao,data,hora,temp_inst,temp_max,temp_min,umid_inst,umid_max,umid_min,pto_orvalho_inst,pto_orvalho_max,pto_orvalho_min,pressao,pressao_max,pressao_min,vento_direcao,vento_vel,vento_rajada,radiacao,precipitacao
0,A305,01/06/2019,0,26.6,26.8,26.4,79,80,78,22.6,22.8,22.5,1009.9,1009.9,1009.2,1.4,104,4.8,-3.51,0.0
1,A305,01/06/2019,23,26.0,26.8,25.9,79,79,75,22.1,22.1,21.7,1010.0,1010.0,1009.2,0.5,121,4.2,-3.54,0.0
2,A305,01/06/2019,22,26.7,27.1,26.6,75,75,72,21.8,21.8,21.5,1009.2,1009.2,1009.0,1.7,94,5.5,-3.54,0.0
3,A305,01/06/2019,21,27.1,28.1,26.9,72,72,66,21.6,21.6,21.2,1009.0,1009.0,1008.4,2.1,91,6.9,70.21,0.0
4,A305,01/06/2019,20,28.1,29.3,28.1,66,67,61,21.2,21.8,20.8,1008.4,1008.4,1007.9,2.9,103,7.4,688.50,0.0


In [43]:
#Correting column's names
data_climate_test_train.columns = ['codigo_estacao','data','hora','temp_inst','temp_max','temp_min','umid_inst','umid_max','umid_min','pto_orvalho_inst','pto_orvalho_max','pto_orvalho_min','pressao','pressao_max','pressao_min','vento_direcao','vento_vel','vento_rajada','radiacao','precipitacao']
data_climate_test_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1464 entries, 0 to 1463
Data columns (total 20 columns):
codigo_estacao      1464 non-null object
data                1464 non-null object
hora                1464 non-null int64
temp_inst           1464 non-null float64
temp_max            1464 non-null float64
temp_min            1464 non-null float64
umid_inst           1464 non-null int64
umid_max            1464 non-null int64
umid_min            1464 non-null int64
pto_orvalho_inst    1464 non-null float64
pto_orvalho_max     1464 non-null float64
pto_orvalho_min     1464 non-null float64
pressao             1464 non-null float64
pressao_max         1464 non-null float64
pressao_min         1464 non-null float64
vento_direcao       1464 non-null float64
vento_vel           1464 non-null int64
vento_rajada        1464 non-null float64
radiacao            1464 non-null float64
precipitacao        1464 non-null float64
dtypes: float64(13), int64(5), object(2)
memory usage: 228.9+ KB


In [44]:
# dif into temp_max and temp_min
diff_temp_test_train = data_climate_test_train.temp_max - data_climate_test_train.temp_min

# diff between umid_max umid_min
diff_umid_test_train = data_climate_test_train.umid_max - data_climate_test_train.umid_min

# diff betwenn dew_max and dew_min
diff_dew_test_train = data_climate_test_train.pto_orvalho_max - data_climate_test_train.pto_orvalho_min

#diff between pressue_max and pressure_min
diff_pressure_test_train = data_climate_test_train.pressao_max - data_climate_test_train.pressao_min

# diff between wind_direction and wind_guest
diff_wind_test_train = data_climate_test_train.vento_rajada - data_climate_test_train.vento_direcao

In [45]:
new_dataset_test_train = pd.DataFrame(data_climate_test_train[['hora','temp_inst','umid_inst','pto_orvalho_inst','pressao','vento_rajada','radiacao','precipitacao','vento_vel']])
new_dataset_test_train.head()

,hora,temp_inst,umid_inst,pto_orvalho_inst,pressao,vento_rajada,radiacao,precipitacao,vento_vel
0,0,26.6,79,22.6,1009.9,4.8,-3.51,0.0,104
1,23,26.0,79,22.1,1010.0,4.2,-3.54,0.0,121
2,22,26.7,75,21.8,1009.2,5.5,-3.54,0.0,94
3,21,27.1,72,21.6,1009.0,6.9,70.21,0.0,91
4,20,28.1,66,21.2,1008.4,7.4,688.50,0.0,103


In [46]:
new_dataset_test_train['diff_temp']     = diff_temp_test_train
new_dataset_test_train['diff_umid']     = diff_umid_test_train
new_dataset_test_train['diff_dew']      = diff_dew_test_train
new_dataset_test_train['diff_pressure'] = diff_pressure_test_train
new_dataset_test_train['diff_wind']     = diff_wind_test_train
new_dataset_test_train.head()

,hora,temp_inst,umid_inst,pto_orvalho_inst,pressao,vento_rajada,radiacao,precipitacao,vento_vel,diff_temp,diff_umid,diff_dew,diff_pressure,diff_wind
0,0,26.6,79,22.6,1009.9,4.8,-3.51,0.0,104,0.4,2,0.3,0.7,3.4
1,23,26.0,79,22.1,1010.0,4.2,-3.54,0.0,121,0.9,4,0.4,0.8,3.7
2,22,26.7,75,21.8,1009.2,5.5,-3.54,0.0,94,0.5,3,0.3,0.2,3.8
3,21,27.1,72,21.6,1009.0,6.9,70.21,0.0,91,1.2,6,0.4,0.6,4.8
4,20,28.1,66,21.2,1008.4,7.4,688.50,0.0,103,1.2,6,1.0,0.5,4.5


In [47]:
#To analyse correlation pearson
new_dataset_test_train.corr()

,hora,temp_inst,umid_inst,pto_orvalho_inst,pressao,vento_rajada,radiacao,precipitacao,vento_vel,diff_temp,diff_umid,diff_dew,diff_pressure,diff_wind
hora,1.000000,0.507792,-0.499000,-0.231222,-0.311503,0.460740,0.445773,-0.058740,-0.383448,0.344601,0.402681,0.406074,0.226158,0.393617
temp_inst,0.507792,1.000000,-0.917384,-0.358297,-0.181019,0.706147,0.856695,-0.207292,-0.508016,0.496841,0.520243,0.717208,0.246622,0.585331
umid_inst,-0.499000,-0.917384,1.000000,0.690023,0.121388,-0.749134,-0.869224,0.235411,0.406688,-0.508857,-0.543056,-0.731616,-0.221322,-0.614664
pto_orvalho_inst,-0.231222,-0.358297,0.690023,1.000000,-0.024330,-0.466777,-0.543500,0.127076,-0.019486,-0.290873,-0.300458,-0.458334,-0.115201,-0.372127
pressao,-0.311503,-0.181019,0.121388,-0.024330,1.000000,-0.117241,-0.099065,0.050652,0.237491,0.067601,0.018654,-0.080807,-0.034971,-0.097301
vento_rajada,0.460740,0.706147,-0.749134,-0.466777,-0.117241,1.000000,0.680531,0.022887,-0.473471,0.440998,0.484879,0.607353,0.213408,0.940019
radiacao,0.445773,0.856695,-0.869224,-0.543500,-0.099065,0.680531,1.000000,-0.106615,-0.203865,0.615958,0.571753,0.810578,0.277870,0.536751
precipitacao,-0.058740,-0.207292,0.235411,0.127076,0.050652,0.022887,-0.106615,1.000000,0.000217,-0.015019,-0.104961,-0.042898,0.113140,0.102715
vento_vel,-0.383448,-0.508016,0.406688,-0.019486,0.237491,-0.473471,-0.203865,0.000217,1.000000,-0.007007,-0.113203,-0.194993,-0.104964,-0.447528
diff_temp,0.344601,0.496841,-0.508857,-0.290873,0.067601,0.440998,0.615958,-0.015019,-0.007007,1.000000,0.836431,0.633721,0.187325,0.375049


In [48]:
new_dataset_test_train.corr()["vento_vel"]

hora               -0.383448
temp_inst          -0.508016
umid_inst           0.406688
pto_orvalho_inst   -0.019486
pressao             0.237491
vento_rajada       -0.473471
radiacao           -0.203865
precipitacao        0.000217
vento_vel           1.000000
diff_temp          -0.007007
diff_umid          -0.113203
diff_dew           -0.194993
diff_pressure      -0.104964
diff_wind          -0.447528
Name: vento_vel, dtype: float64

In [49]:
#x = set_test_train[['hora','temp_inst','umid_inst','pto_orvalho_inst','pressao','vento_rajada','radiacao','precipitacao','diff_temp','diff_umid','diff_dew','diff_pressure','diff_wind']]
x_one_moth = new_dataset_test_train[['hora','temp_inst','umid_inst','vento_rajada','diff_wind']]
y_one_moth = new_dataset_test_train[['vento_vel']]


print('X:{}'.format(x_one_moth.shape))
print('Y:{}'.format(y_one_moth.shape))

x = x_one_moth[:732]
y = y_one_moth[732:1464]


print('X:{}'.format(x.shape))
print('Y:{}'.format(y.shape))


X:(1464, 5)
Y:(1464, 1)
X:(732, 5)
Y:(732, 1)


In [57]:
x_numpy = x.to_data
y_numpy = y.to_data

In [74]:
x_numpy

array(<bound method DataFrame.to_numpy of      hora  temp_inst  umid_inst  vento_rajada  diff_wind
0       0       26.6         79           4.8        3.4
1      23       26.0         79           4.2        3.7
2      22       26.7         75           5.5        3.8
3      21       27.1         72           6.9        4.8
4      20       28.1         66           7.4        4.5
..    ...        ...        ...           ...        ...
727    16       30.0         50          12.5        8.0
728    17       29.7         56          12.5        8.3
729    18       29.2         61          10.3        6.3
730    19       28.6         64           9.3        5.5
731    21       26.4         73           7.8        5.4

[732 rows x 5 columns]>, dtype=object)

In [59]:
y_numpy

<bound method DataFrame.to_numpy of       vento_vel
732         112
733         105
734         110
735         114
736         118
...         ...
1459         99
1460        106
1461        116
1462         97
1463        105

[732 rows x 1 columns]>

In [63]:
# Train and test elm network
elmk = elm.ELMKernel()
elmk.search_param(x_numpy, cv="kfold", of="accuracy", eval=10)
tr_set, te_set = elm.split_sets(x_numpy, training_percent=.8, perm=True)
tr_result = elmk.train(tr_set)
te_result = elmk.test(te_set)

print(te_result.get_accuracy)

elmk
##### Start search #####


/usr/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


TypeError: object of type 'method' has no len()